In [95]:
import numpy as np

In [451]:
def generate_TF_inner(arr, n):
        if n == 0:
            return arr
        else:
            new_arr = []
            for i in arr:
                for possible_value in (True, False):
                    new_i = i.copy()
                    new_i.append(possible_value)
                    new_arr.append(new_i)
            if (len(new_arr) == 0):
                new_arr = [[True],[False]]
            return generate_TF_inner(new_arr, n-1)
                    
# generate all possible values of True, False for n variables
def generate_TF(n):
    arr = []
    ans = generate_TF_inner(arr, n)
    return ans

In [707]:
def simplifyString(string):
    return string.replace(" ", "")

Парсим выражение.
Есть операторы. Есть формулы. Формулы - это либо атомы, либо выражения. Объединение операторов и других формул даёт новую формулу.

Видим выражение (A&B)|(!A&(B|C))&D

1. Атом - инстанс класса, который хранит в себе номер аргумента из комбинации значений всех атомов в рамках одной интерпретации.
2. Делаем сет всех атомов.
3. Дробим исходное выражение на формулы. 
    1. Сначала отсекаем части по операторам (то есть что между скобок) - в рамках одного уровня заносим все встреченные операции. ['|', '&']
    2. Вместо скобок создаём формулы. [(A&B), (!A&(B|C), D]
    4. Эти штуки рекурсивно преобразуем по той же схеме. 
    5. После преобразования каждая штука должна возвратить формулу куда можно подставить комбинацию значений.
    6. Полученные формулы объединеям в одну с учетом стека операций и возвращаем.
4. Получится большая лямбда, которая принимает комбинацию всех значений.
5. Делаем таблицу.



In [683]:
class Atom:
    def __init__(self, name):
        self.name = name
    def substitute(self, args):
        print(f"substitute {args[self.name]} at {self}")
        return args[self.name]
    def __str__(self):
        return "Atom " + self.name
    def __repr__(self):
        return "Atom " + self.name

class Op:
    def __init__(self, op_string, unary):
        self.op_string = op_string
        self.unary = unary
    def __str__(self):
        return self.op_string
    def __repr__(self):
        return self.op_string
    def op_on(self, *lambdas):
        pass
    def identify(self, expr):
        if (expr.find(self.op_string) == 0):
            return True
        else:
            return False

class UnaryNeg(Op):
    def __init__(self):
        super().__init__("!", True)
    def op_on(self, *lambdas):
        return lambda x: not lambdas[0](x)
class AndOp(Op):
    def __init__(self):
        super().__init__("&", False)
    def op_on(self, *lambdas):
        return lambda x: lambdas[0](x) and lambdas[1](x)
class OrOp(Op):
    def __init__(self):
        super().__init__("|", False)
    def op_on(self, *lambdas):
        return lambda x: lambdas[0](x) or lambdas[1](x)
class ImplicatOp(Op):
    def __init__(self):
        super().__init__("->", False)
    def op_on(self, *lambdas):
        def implicat(x):
            a = lambdas[0](x)
            b = lambdas[1](x)
            if a and not b:
                return False
            else:
                return True
        return implicat
class EquivOp(Op):
    def __init__(self):
        super().__init__("~", False)
    def op_on(self, *lambdas):
        def equive(x):
            a = lambdas[0](x)
            b = lambdas[1](x)
            if (a and b) or (not a and not b):
                return True
            else:
                return False
        return equive
# try to identify the op in the given string and return it
def identify_op(expression):
    ops_set = {UnaryNeg(): True, AndOp(): True, OrOp(): True, ImplicatOp(): True, EquivOp(): True}
    for op in ops_set:
        if (op.identify(expression)):
            return op
    return None

In [467]:
def create_atom_set(expression):
    atom_set = {}
    for i in expression:
        if i not in atom_set and re.match('[a-zA-Z]', i):
            atom_set[i] = Atom(i)
            print(i)
    return atom_set

def get_var_names(expression):
    names = []
    for i in expression:
        if i not in names and re.match('[a-zA-Z]', i):
            names.append(i)
    return names

In [721]:
import pandas as pd
def true_table_for(fun, var_names):
    table = []
    result_keyword = "result"
    all_combs = generate_TF(len(var_names))
    for i in all_combs:
        new_comb = {}
        for index, char in enumerate(var_names):
            new_comb[char] = i[index]
        result = fun(new_comb)
        new_comb[result_keyword] = result
        table.append(new_comb)
    frame = pd.DataFrame(table)
    return frame
def full_solve(expr):
    fun = lambda_for_adv(expr, None)
    return true_table_for(fun, get_var_names(expr))

In [722]:
def collect_as_labmda(ops_stack, lambda_stack):
    lambda_stack_copy = lambda_stack.copy()
    current_lambda = None
    if (len(lambda_stack_copy) == 1):
        return lambda_stack_copy[0]
    for i in range(0, len(ops_stack)):
        if (current_lambda == None):
            lam1 = lambda_stack_copy[0]
            lam2 = lambda_stack_copy[1]
            print(f"lambdas to concat {lam1}, {lam2}")
            current_lambda = ops_stack[i].op_on(lam1, lam2)
            lambda_stack_copy = lambda_stack_copy[2:]
        else:
            lam1 = current_lambda
            lam2 = lambda_stack_copy[0]
            print(f"lambdas to concat {lam1}, {lam2}")
            current_lambda = ops_stack[i].op_on(current_lambda, lam2)
            lambda_stack_copy = lambda_stack_copy[1:]
    return current_lambda

def lambda_for_adv(expression, atom_set):
    # build variables dictionary
    if (atom_set == None):
        expression = simplifyString(expression)
        atom_set = create_atom_set(expression)
    ops_stack = []
    parenth_stack = []
    lambdas_stack = []        
    begin = 0
    for i, char in enumerate(expression):
        if (char == '('):
            if (len(parenth_stack) == 0):
                begin = i
            parenth_stack.append('(')
        elif (char == ')'):
            if (parenth_stack[-1] == '(' and len(parenth_stack) == 1):
                inner_expr = expression[begin+1:i]
                print(f"inner parse: {begin+1}, {i}")
                # parse inner expression and build lambda
                inner_lambda = lambda_for_adv(inner_expr, atom_set)
                lambda_to_append = inner_lambda
                if (len(ops_stack) > 0):
                    last_op = ops_stack[-1]
                    while (last_op != None and last_op.unary):
                        lambda_to_append = last_op.op_on(lambda_to_append)
                        if (last_op.unary):
                            ops_stack.pop()
                        if (len(ops_stack) > 0):
                            last_op = ops_stack[-1]
                        else:
                            last_op = None
                lambdas_stack.append(lambda_to_append)
            parenth_stack.pop()
        elif (len(parenth_stack) == 0):
            # now need to understand what kind of sumbol encountered
            identified_op = identify_op(expression[i:])
            if (identified_op != None):
                print(f"identified op: {identified_op}")
                ops_stack.append(identified_op)
            elif re.match('[a-zA-Z]', char):
                # if a char than just collect as lambda
                lambda_to_append = atom_set[char].substitute
                if (len(ops_stack) > 0):
                    last_op = ops_stack[-1]
                    while (last_op != None and last_op.unary):
                        lambda_to_append = last_op.op_on(lambda_to_append)
                        if (last_op.unary):
                            ops_stack.pop()
                        if (len(ops_stack) > 0):
                            last_op = ops_stack[-1]
                        else:
                            last_op = None
                lambdas_stack.append(lambda_to_append)
    # now the ops and lambdas must be filled
    print(ops_stack, lambdas_stack)
    full_lambda = collect_as_labmda(ops_stack, lambdas_stack)
    return full_lambda

In [725]:
solution = full_solve("!A|(Q&B)")
print(solution)

A
Q
B
identified op: !
identified op: |
inner parse: 4, 7
identified op: &
[&] [<bound method Atom.substitute of Atom Q>, <bound method Atom.substitute of Atom B>]
lambdas to concat <bound method Atom.substitute of Atom Q>, <bound method Atom.substitute of Atom B>
[|] [<function UnaryNeg.op_on.<locals>.<lambda> at 0x7f919d8285e0>, <function AndOp.op_on.<locals>.<lambda> at 0x7f919d419040>]
lambdas to concat <function UnaryNeg.op_on.<locals>.<lambda> at 0x7f919d8285e0>, <function AndOp.op_on.<locals>.<lambda> at 0x7f919d419040>
substitute True at Atom A
substitute True at Atom Q
substitute True at Atom B
substitute True at Atom A
substitute True at Atom Q
substitute False at Atom B
substitute True at Atom A
substitute False at Atom Q
substitute True at Atom A
substitute False at Atom Q
substitute False at Atom A
substitute False at Atom A
substitute False at Atom A
substitute False at Atom A
       A      Q      B  result
0   True   True   True    True
1   True   True  False   False
2  